In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import NN

In [2]:
import dataset_utils
import importlib

In [3]:
importlib.reload(dataset_utils)

<module 'dataset_utils' from 'd:\\University\\P4P\\P4P-mmWave\\MMWave_Radar_Human_Tracking_and_Fall_detection\\dataset_utils.py'>

In [4]:
DATA_DIR = "2_class_data"
NUM_POINTS = 100
NUM_CLASSES = 2
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: sitting
processing class: standing


In [5]:
print(test_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [6]:
print(train_points)

[[[ 1.5796e-01  5.6494e-01  9.4531e-01  5.0098e-01  2.9000e+02]
  [ 1.3977e-01  5.6494e-01  9.2188e-01  5.0098e-01  2.9000e+02]
  [ 2.4072e-01  1.1835e-01  8.3691e-01  3.7573e-01  3.1800e+02]
  ...
  [ 1.0000e+00  3.5938e-01  6.0010e-01  3.7573e-01  3.1000e+02]
  [ 2.5098e-01  7.2510e-01  6.8115e-01  1.2524e-01  2.9500e+02]
  [ 2.4072e-01  1.9080e-01  7.6270e-01  2.5049e-01  2.6300e+02]]

 [[ 3.8867e-01  4.7070e-01  1.0000e+00 -5.0098e-01  2.3700e+02]
  [ 0.0000e+00  2.5513e-01  5.5615e-01  5.0098e-01  3.0800e+02]
  [ 7.0020e-01  8.9697e-01  6.6650e-01 -1.2524e-01  1.3500e+02]
  ...
  [ 3.9160e-01  3.7744e-01  0.0000e+00 -2.5049e-01  1.8900e+02]
  [ 1.0000e+00  1.2598e-01  5.2490e-01 -2.5049e-01  1.8000e+02]
  [ 0.0000e+00  3.3813e-01  8.4167e-02 -3.7573e-01  2.2500e+02]]

 [[ 4.6484e-01  0.0000e+00  9.1455e-01 -5.0098e-01  3.3700e+02]
  [ 1.0000e+00  7.5000e-01  5.5713e-01  2.5049e-01  2.5800e+02]
  [ 6.5381e-01  1.0000e+00  4.4116e-01  2.5049e-01  3.6700e+02]
  ...
  [ 4.8584e-02  7.

In [7]:
from tensorflow import data as tf_data
import keras
#import PointNET

keras.utils.set_random_seed(seed=42)

In [8]:
train_size = 0.8
dataset = tf_data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf_data.Dataset.from_tensor_slices((test_points, test_labels))
train_dataset_size = int(len(dataset) * train_size)

In [9]:
dataset = dataset.shuffle(len(train_points)).map(NN.augment)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)
train_dataset = dataset.take(train_dataset_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_dataset_size).batch(BATCH_SIZE)

In [10]:
inputs = keras.Input(shape=(NUM_POINTS, 5))

In [11]:
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

#model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape))
#model.add(layers.MaxPooling1D(pool_size=2))
#model.add(Dropout(0.25))
#model.add(BatchNormalization())

# # 2. LSTM layer
# model.add(layers.LSTM(50, return_sequences=True))
# model.add(layers.LSTM(50))

# 3. Dense layers
#model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#model.add(layers.Dense(1, activation='linear'))  # Output layer (you can adjust the output size and activation)
model.add(layers.Dense(1, activation='softmax'))


c:\Users\Yang Qian\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, None, 64)       │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 64)       │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 1)        │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,505 (21.50 KB)

 Trainable params: 5,377 (21.00 KB)

 Non-trainable params: 128 (512.00 B)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
# training
history = model.fit(train_points, train_labels, epochs=30, batch_size=16, validation_data=(test_points, test_labels), verbose=1, callbacks=[early_stopping])

Epoch 1/30


c:\Users\Yang Qian\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 49, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.5025 - mae: 0.5025 - val_loss: 0.4964 - val_mae: 0.4964
Epoch 2/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5025 - mae: 0.5025 - val_loss: 0.4964 - val_mae: 0.4964
Epoch 3/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5025 - mae: 0.5025 - val_loss: 0.4964 - val_mae: 0.4964
Epoch 4/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5025 - mae: 0.5025 - val_loss: 0.4964 - val_mae: 0.4964


In [15]:
test_loss, test_acc = model.evaluate(test_points, test_labels, verbose=1)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7749 - mae: 0.7749 


In [16]:
print(test_acc)

0.49642857909202576


In [17]:
# pickle.dump(model, open("model.pkl", "wb"))

In [18]:
# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

# Flatten the output
model.add(layers.Flatten())

# Summary of the CNN model
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, None, 64)       │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, None, 64)       │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, None)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,888 (54.25 KB)

 Trainable params: 13,632 (53.25 KB)

 Non-trainable params: 256 (1.00 KB)

In [19]:
import numpy as np

# Assuming sequence_length = 100 for this example
sequence_length = 100
#X = np.random.random((10, sequence_length, 5))  # Example data

# Extract features using CNN
X_Train_Features = model.predict(train_points)
X_Test_Features = model.predict(test_points)

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [20]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

svm_classifer = svm.SVC(kernel='linear')
svm_classifer.fit(X_Train_Features, train_labels)

y_pred = svm_classifer.predict(X_Test_Features)

accuracy = accuracy_score(test_labels, y_pred)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 87.14


In [21]:
print(len(X_Train_Features))
print(len(X_Test_Features))
print(len(train_labels))
print(len(test_labels))

1114
280
1114
280


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
svc=svm.SVC(probability=True)
print("The training of the model is started, please wait for while as it may take few minutes to complete")
model=GridSearchCV(svc,param_grid, refit = True, verbose = 4, n_jobs=-1)
model.fit(x_train,y_train)
print('The Model is trained well with the given images')
model.best_params_

pickle.dump(model,open('SVM_model.p', 'wb'))
y_pred=model.predict(x_test)

print(f"The models is {accuracy_score(y_pred,y_test)*100}% accurate")